In [1]:
import numpy as np
import tkinter as tk
from tkinter import filedialog
# import rcm

# a = rcm.B(0,1,2,3,0,7,8,6,8,9,15,5)
# print(a)

root = tk.Tk()
root.withdraw()

# try: 
file_path = filedialog.askopenfilename(
  title="lel",
  initialdir=".",
  filetypes=(
      # ("jpeg files", "*.jpg"),
      ("csv files", "*.csv"),
      ("xyz files", "*.xyz"),
      # ("gif files", "*.gif*"),
      # ("png files", "*.png")
    )
  )
# except Exception, e:
#   raise tkMessageBox.showerror('Error Loading Chart',
#         'Unable to open file') 

print(file_path)


In [4]:
import pandas as pd

In [7]:
pd.read_csv(file_path,header=None)

,0,1,2,3,4,5,6,7
0,315,325,428,427,368,364,230,231
1,271,282,429,430,369,365,242,241
2,293,260,438,437,378,374,252,253
3,303,227,439,440,379,375,263,264
4,248,347,448,447,388,384,275,274
5,326,337,449,450,389,385,285,286
6,238,237,458,457,398,394,297,296
7,281,304,459,460,399,395,307,308
8,336,292,468,467,408,404,319,318
9,348,249,469,470,409,405,330,329


In [2]:
# try: 
file_path = filedialog.askopenfilename(
  title="lel",
  initialdir=".",
  filetypes=(
      # ("jpeg files", "*.jpg"),
      ("xyz files", "*.xyz"),
      # ("gif files", "*.gif*"),
      # ("png files", "*.png")
    )
  )
# except Exception, e:
#   raise tkMessageBox.showerror('Error Loading Chart',
#         'Unable to open file') 

print(file_path)


C:/MJ_python/rcm-python/cp6t6_0p_blyp35_geom.xyz


In [3]:
# from openbabel import openbabel

In [5]:
from rdkit import Chem

In [15]:
a = Chem.rdmolfiles.MolFromXYZFile(file_path)

In [22]:
b = Chem.MolToXYZBlock(a)

In [37]:
print(b)

480

C     12.834420   -0.431648    1.263983
C     13.087348   -1.095274    2.516019
C     13.080758   -0.132733    3.471963
C     12.782510    1.115903    2.809385
N     12.655821    0.898976    1.462114
H     13.278680   -2.148919    2.630685
H     13.268689   -0.251114    4.524947
C     12.625773    2.352190    3.456012
C     12.303551    3.558170    2.810769
C     12.027016    4.806234    3.482818
N     12.142265    3.721661    1.459596
C     11.687678    5.704844    2.523805
H     12.079604    4.967529    4.545609
C     11.776003    5.014922    1.264280
H     11.420065    6.740939    2.645418
C     11.538329    5.614799    0.006059
C     11.799137    5.017435   -1.248456
C     11.724284    5.709996   -2.508171
N     12.173023    3.726281   -1.440765
C     12.076091    4.813870   -3.463929
H     11.453695    6.745103   -2.631236
C     12.345539    3.564741   -2.789904
H     12.138332    4.975883   -4.526045
C     12.667224    2.359602   -3.436118
C     12.805991    1.119015   -2.79

In [29]:
from rdkit.Chem import PandasTools

In [34]:
import pandas as pd

In [35]:
pd.DataFrame(b)

ValueError: DataFrame constructor not properly called!

In [26]:
b

'480\n\nC     12.834420   -0.431648    1.263983\nC     13.087348   -1.095274    2.516019\nC     13.080758   -0.132733    3.471963\nC     12.782510    1.115903    2.809385\nN     12.655821    0.898976    1.462114\nH     13.278680   -2.148919    2.630685\nH     13.268689   -0.251114    4.524947\nC     12.625773    2.352190    3.456012\nC     12.303551    3.558170    2.810769\nC     12.027016    4.806234    3.482818\nN     12.142265    3.721661    1.459596\nC     11.687678    5.704844    2.523805\nH     12.079604    4.967529    4.545609\nC     11.776003    5.014922    1.264280\nH     11.420065    6.740939    2.645418\nC     11.538329    5.614799    0.006059\nC     11.799137    5.017435   -1.248456\nC     11.724284    5.709996   -2.508171\nN     12.173023    3.726281   -1.440765\nC     12.076091    4.813870   -3.463929\nH     11.453695    6.745103   -2.631236\nC     12.345539    3.564741   -2.789904\nH     12.138332    4.975883   -4.526045\nC     12.667224    2.359602   -3.436118\nC     12

In [21]:
print(Chem.MolToXYZBlock(a))

480

C     12.834420   -0.431648    1.263983
C     13.087348   -1.095274    2.516019
C     13.080758   -0.132733    3.471963
C     12.782510    1.115903    2.809385
N     12.655821    0.898976    1.462114
H     13.278680   -2.148919    2.630685
H     13.268689   -0.251114    4.524947
C     12.625773    2.352190    3.456012
C     12.303551    3.558170    2.810769
C     12.027016    4.806234    3.482818
N     12.142265    3.721661    1.459596
C     11.687678    5.704844    2.523805
H     12.079604    4.967529    4.545609
C     11.776003    5.014922    1.264280
H     11.420065    6.740939    2.645418
C     11.538329    5.614799    0.006059
C     11.799137    5.017435   -1.248456
C     11.724284    5.709996   -2.508171
N     12.173023    3.726281   -1.440765
C     12.076091    4.813870   -3.463929
H     11.453695    6.745103   -2.631236
C     12.345539    3.564741   -2.789904
H     12.138332    4.975883   -4.526045
C     12.667224    2.359602   -3.436118
C     12.805991    1.119015   -2.79

In [ ]:

rdkit.Chem.rdmolfiles.MolFromXYZFile((str)xyzFileName) → Mol :

    Construct a molecule from an XYZ file.

        ARGUMENTS:

                xyzname: name of the file to read

        RETURNS:

            a Mol object, None on failure.

    C++ signature :

        RDKit::ROMol* MolFromXYZFile(char const*)



In [17]:
pd.read_table(file_path,header=None,delimiter=' ')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 2, saw 6


In [2]:
import rcm

a = rcm.Bz(0,1,2,3,0,7,8,6,8,9,15,5)
print(a)
a = rcm.Bx(0,1,2,3,0,7,8,6,8,9,15,5)
print(a)
a = rcm.By(0,1,2,3,0,7,8,6,8,9,15,5)
print(a)

-1.9532285990392415
0.7812914396156966
-0.0


In [3]:
a = rcm.B(0,1,2,3,0,7,8,6,8,9,15,5)
print(a)

(0.7812914396156966, -0.0, -1.9532285990392415)


In [8]:
type(a)

numpy.float64